<a href="https://colab.research.google.com/github/PRAN20/Plant-Disease-Detection/blob/main/alexnet_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


loading model

In [ ]:
import keras
model= keras.models.load_model("/content/drive/MyDrive/Alex.hdf5")

viewing model layers

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 27, 27, 96)       384       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                             
                                                      

selecting model layers from 0 to n-1 and initializing it to new_model / tried pop function but wasn't working

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.Sequential(model.layers[:-1])

checking new_model layers

In [ ]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_8 (Batc  (None, 27, 27, 96)       384       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                             
                                                        

getting dataset

In [ ]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163229, done.
remote: Total 163229 (delta 0), reused 0 (delta 0), pack-reused 163229
Receiving objects: 100% (163229/163229), 2.00 GiB | 30.11 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (182401/182401), done.


checking if prediction is giving right number of features

In [ ]:
import cv2
import numpy as np
img1= cv2.imread("/content/PlantVillage-Dataset/data_distribution_for_SVM/train/0/007cb8ca-b40c-46a2-9b7b-19f5b43b0e78.JPG",1)
img1 = cv2.resize(img1,(224,224))
img1= np.expand_dims(img1,axis=0)
(new_model.predict(img1/255))[0].flatten().shape

(1000,)

passing all images from all folders in train and storing predicted features by new_model into train_imgs array / storing folder name as label in train_labels array.

In [ ]:
import os
import cv2
import numpy as np
train_imgs=[]
train_labels=[]
n=1
for folder in os.listdir("/content/PlantVillage-Dataset/data_distribution_for_SVM/train"):
  for imgs in os.listdir("/content/PlantVillage-Dataset/data_distribution_for_SVM/train/"+folder):
    img1_path = cv2.imread("/content/PlantVillage-Dataset/data_distribution_for_SVM/train/"+folder+"/"+imgs,1)
    img1 = cv2.resize(img1_path,(224,224))
    img1= np.expand_dims(img1,axis=0)
    img1_embedding = new_model.predict(img1/255)[0].flatten()
    train_imgs.append(img1_embedding)
    train_labels.append(int(folder))

same thing with test folder

In [ ]:
import os
import cv2
import numpy as np
test_imgs=[]
test_labels=[]
n=1
for folder in os.listdir("/content/PlantVillage-Dataset/data_distribution_for_SVM/test"):
  for imgs in os.listdir("/content/PlantVillage-Dataset/data_distribution_for_SVM/test/"+folder):
    img1_path = cv2.imread("/content/PlantVillage-Dataset/data_distribution_for_SVM/test/"+folder+"/"+imgs,1)
    img1 = cv2.resize(img1_path,(224,224))
    img1= np.expand_dims(img1,axis=0)
    img1_embedding = new_model.predict(img1/255)[0].flatten()
    test_imgs.append(img1_embedding)
    test_labels.append(int(folder))

converting to numpy array

In [ ]:
train_imgs= np.array(train_imgs)
train_labels= np.array(train_labels)

checking if we got correct array as planned

In [ ]:
train_imgs.shape, train_labels.shape

((8751, 1000), (8751,))

saving train_imgs to X_train.csv

In [ ]:
import numpy
a = numpy.asarray(train_imgs)
numpy.savetxt("X_train.csv", a, delimiter=",")

saving train_labels to y_train.csv

In [ ]:
a = numpy.asarray(train_labels)
numpy.savetxt("y_train.csv", a, delimiter=",")

converting test images and labels to numpy array

In [ ]:
test_imgs= np.array(test_imgs)
test_labels= np.array(test_labels)
test_imgs.shape, test_labels.shape

((10547, 1000), (10547,))

saving test_imgs to X_test and test_labels to y_test.csv

In [ ]:
a = numpy.asarray(test_imgs)
numpy.savetxt("X_test.csv", a, delimiter=",")
a = numpy.asarray(test_labels)
numpy.savetxt("y_test.csv", a, delimiter=",")

In [ ]:
from zipfile import ZipFile
import os
  
def get_all_file_paths(directory):
    file_paths = []
  
    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    return file_paths        
  
def main():
    directory = '/content/CSV-FILES'
    file_paths = get_all_file_paths(directory)
    print('Following files will be zipped:')
    for file_name in file_paths:
        print(file_name)
  
    with ZipFile('my_python_files.zip','w') as zip:
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
  
if __name__ == "__main__":
    main()

Following files will be zipped:
/content/CSV-FILES/X_test.csv
/content/CSV-FILES/X_train.csv
/content/CSV-FILES/y_test.csv
/content/CSV-FILES/y_train.csv
All files zipped successfully!
